In [ ]:
import pandas as pd
import numpy as np
import pickle
from tensorflow import keras
from data_setup import ZRI_format

In [ ]:
#open dataframe
file = open('pickles/ZRI_filtered.p','rb')
ZRI_filtered = pickle.load(file)

In [ ]:
#try: keras.preprocessing.timeseries_dataset_from_array
#ZRI = ZRI_format(ZRI_filtered,window_size = 6, future_time = 12)
data_use = ZRI_filtered[[x for x in ZRI_filtered.columns if ('20' in x)]]
ZRI_data = keras.preprocessing.timeseries_dataset_from_array(data_use.iloc[:,:-6],data_use.iloc[:,6:],sequence_length = 6)
for batch in ZRI_data:
    inputs, targets = batch


In [ ]:
dataset_val = keras.preprocessing.timeseries_dataset_from_array(
    x_val,
    y_val,
    sequence_length=sequence_length,
    sampling_rate=step,
    batch_size=batch_size,
)

In [ ]:
inputs = keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2]))
lstm_out = keras.layers.LSTM(32)(inputs)
outputs = keras.layers.Dense(1)(lstm_out)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=keras.optimizers.Adam(), loss="mse")
model.summary()

In [ ]:
path_checkpoint = "model_checkpoint.h5"
es_callback = keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=5)

modelckpt_callback = keras.callbacks.ModelCheckpoint(
    monitor="val_loss",
    filepath=path_checkpoint,
    verbose=1,
    save_weights_only=True,
    save_best_only=True,
)

history = model.fit(
    dataset_train,
    epochs=epochs,
    validation_data=dataset_val,
    callbacks=[es_callback, modelckpt_callback],
)

In [ ]:
def visualize_loss(history, title):
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]
    epochs = range(len(loss))
    plt.figure()
    plt.plot(epochs, loss, "b", label="Training loss")
    plt.plot(epochs, val_loss, "r", label="Validation loss")
    plt.title(title)
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()


visualize_loss(history, "Training and Validation Loss")

In [ ]:
feature_columns = [x for x in ZRI.columns if 'minus' in x]
ZRI = ZRI.dropna()

In [ ]:
test_year = 2019
training_data = ZRI[ZRI.Year < test_year]
final_test_data = ZRI[ZRI.Year >= test_year]

In [ ]:
X_train, y_train = training_data[feature_columns], training_data['Target_ZRI']

In [ ]:
y_train